# Final Data Science Capstone

### Data Section

In order to achieve our objectives stated in the Introduction. We use the postcode areas dataset in the folowing link https://en.wikipedia.org/wiki/London_postal_district. We scrap using pandas.

The postcode areas in london are  N, NW, SW, SE, W, WC, E and EC postcode areas. For each postcode area, we collect data on all the postcode districts, post town, coverage and local authority area(s) in the area. We merge all the 8 postcode areas to generate our final dataset with all coverages(distincts) given.

We remove the data points where the local authgority area(s) is non-geographic and other None values.

We use the Geopy library to find the location data for each district in our dataset.

Finally, we use the Foursquare API to explore the venues around each district using the location data provided by Geopy.

### Import all the necessary libaries

In [1]:
#import necessary libaries
!pip install geocoder
import geocoder # import geocoder

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!pip install geopy  # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


The eight postcode areas are given as follows

In [2]:
postcode_areas = [ 'N', 'NW', 'SW', 'SE', 'W', 'WC', 'E' , 'EC']
tables = []

for item in postcode_areas:
    
    if item != 'EC':
        url = 'https://en.wikipedia.org/wiki/{}_postcode_area'.format(item)
        tables_pd = pd.read_html(url)
        tables.append(tables_pd[1])
        
    elif item == 'EC':
        url = 'https://en.wikipedia.org/wiki/{}_postcode_area'.format(item)
        tables_pd = pd.read_html(url)
        tables.append(tables_pd[2])
    else:
        pass

Merge all the eight tables from each postcode areas together

In [3]:
result_df = pd.concat(tables)

In [4]:
result_df.head()

,Postcode district,Post town,Coverage,Local authority area(s)
0,N1,LONDON,"Northern head district: Barnsbury (part), Cano...","Hackney, Islington, Camden"
1,N1C,LONDON,Kings Cross Central,Camden
2,N1P,LONDON,PO boxes in N1 and N1C,non-geographic
3,N2,LONDON,"East Finchley district: East Finchley, Fortis ...","Barnet, Haringey"
4,N3,LONDON,"Finchley district: Finchley, Church End, Finch...",Barnet


In [5]:
print('There are {} rows and {} columns in our dataset'.format(result_df.shape[0], result_df.shape[1]))

There are 184 rows and 4 columns in our dataset


Remove all data points where local authority area(s) is non-geographic.

In [6]:
result_df = result_df[result_df['Local authority area(s)'] != 'non-geographic']
result_df

,Postcode district,Post town,Coverage,Local authority area(s)
0,N1,LONDON,"Northern head district: Barnsbury (part), Cano...","Hackney, Islington, Camden"
1,N1C,LONDON,Kings Cross Central,Camden
3,N2,LONDON,"East Finchley district: East Finchley, Fortis ...","Barnet, Haringey"
4,N3,LONDON,"Finchley district: Finchley, Church End, Finch...",Barnet
5,N4,LONDON,"Finsbury Park district: Finsbury Park, Manor H...","Haringey, Islington, Hackney"
6,N5,LONDON,"Highbury district: Highbury, Highbury Fields","Islington, Hackney"
7,N6,LONDON,"Highgate district: Highgate, Hampstead Heath (...","Camden, Haringey, Islington"
8,N7,LONDON,"Holloway district: Holloway, Barnsbury (part),...","Islington, Camden"
9,N8,LONDON,"Hornsey district: Hornsey, Crouch End, Haringe...","Haringey, Islington"
10,N9,LONDON,"Lower Edmonton district: Lower Edmonton, Edmon...",Enfield


In [7]:
result_df = result_df.drop('Local authority area(s)', axis = 1).dropna()

In [8]:
result_df['Coverage'] = result_df['Coverage'].map(lambda name: name.split(':')[0])
result_df = result_df.reset_index(drop = True)
result_df

,Postcode district,Post town,Coverage
0,N1,LONDON,Northern head district
1,N1C,LONDON,Kings Cross Central
2,N2,LONDON,East Finchley district
3,N3,LONDON,Finchley district
4,N4,LONDON,Finsbury Park district
5,N5,LONDON,Highbury district
6,N6,LONDON,Highgate district
7,N7,LONDON,Holloway district
8,N8,LONDON,Hornsey district
9,N9,LONDON,Lower Edmonton district
